In [2]:
import requests
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup

#### Web Scraping

In this Jupyter, we are going to Web Scrape Esportsearnings web page to obtain all the tourneys played
in the last 10 years.

In [3]:
url = "https://www.esportsearnings.com/history/2012-01/games"

In [4]:
res1 = requests.get(url)
res1.status_code

200

In [5]:
sopa = BeautifulSoup(res1.content, 'html.parser') 

In [6]:
raw_data = sopa.find_all("table", {"class": "detail_list_table"})[0]

In [7]:
len(raw_data.find_all("tr")[0].find_all("td"))

5

In [8]:
dicc = {"posicion": [],
        "juego": [],
        "dinero": [],
        "jugadores": [],
        "torneos": [],
        "fecha": []
        }

for i in raw_data.find_all("tr"):
    dicc["posicion"].append(i.find_all("td")[0].getText())
    dicc["juego"].append(i.find_all("td")[1].getText())
    dicc["dinero"].append(i.find_all("td")[2].getText())
    dicc["jugadores"].append(i.find_all("td")[3].getText())
    dicc["torneos"].append(i.find_all("td")[4].getText())

In [10]:
meses = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
años = list(range(2012,2023))
dicc = {"posicion": [],
        "juego": [],
        "dinero": [],
        "jugadores": [],
        "torneos": [],
        "fecha": []
        }

for año in años:
    for mes in meses:
        url = f"https://www.esportsearnings.com/history/{año}-{mes}/games"
        print(url)
        res1 = requests.get(url)
        sopa = BeautifulSoup(res1.content, 'html.parser')
        raw_data = sopa.find_all("table", {"class": "detail_list_table"})[0]
        for i in raw_data.find_all("tr"):
            dicc["posicion"].append(i.find_all("td")[0].getText())
            dicc["juego"].append(i.find_all("td")[1].getText())
            dicc["dinero"].append(i.find_all("td")[2].getText())
            dicc["jugadores"].append(i.find_all("td")[3].getText())
            dicc["torneos"].append(i.find_all("td")[4].getText())
            dicc["fecha"].append(f"{mes}-{año}")

https://www.esportsearnings.com/history/2012-01/games
https://www.esportsearnings.com/history/2012-02/games
https://www.esportsearnings.com/history/2012-03/games
https://www.esportsearnings.com/history/2012-04/games
https://www.esportsearnings.com/history/2012-05/games
https://www.esportsearnings.com/history/2012-06/games
https://www.esportsearnings.com/history/2012-07/games
https://www.esportsearnings.com/history/2012-08/games
https://www.esportsearnings.com/history/2012-09/games
https://www.esportsearnings.com/history/2012-10/games
https://www.esportsearnings.com/history/2012-11/games
https://www.esportsearnings.com/history/2012-12/games
https://www.esportsearnings.com/history/2013-01/games
https://www.esportsearnings.com/history/2013-02/games
https://www.esportsearnings.com/history/2013-03/games
https://www.esportsearnings.com/history/2013-04/games
https://www.esportsearnings.com/history/2013-05/games
https://www.esportsearnings.com/history/2013-06/games
https://www.esportsearnings.

In [12]:
df = pd.DataFrame(dicc)
df.head(3)

,posicion,juego,dinero,jugadores,torneos,fecha
0,1.,StarCraft II,"$89,399.27",105 Players,53 Tournaments,01-2012
1,2.,League of Legends,"$53,334.98",117 Players,22 Tournaments,01-2012
2,3.,Defense of the Ancients,"$52,313.09",25 Players,2 Tournaments,01-2012


In [16]:
df.to_csv("Tourneys_raw")

#### Data Wrangling & Data Cleaning

In [29]:
df = pd.read_csv("Tourneys_raw")
df.drop(["Unnamed: 0"], axis=1,inplace=True)

In [18]:
df.head()

,posicion,juego,dinero,jugadores,torneos,fecha
0,1.0,StarCraft II,"$89,399.27",105 Players,53 Tournaments,01-2012
1,2.0,League of Legends,"$53,334.98",117 Players,22 Tournaments,01-2012
2,3.0,Defense of the Ancients,"$52,313.09",25 Players,2 Tournaments,01-2012
3,4.0,Counter-Strike,"$51,938.70",75 Players,4 Tournaments,01-2012
4,5.0,Heroes of Newerth,"$34,345.67",16 Players,3 Tournaments,01-2012


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6898 entries, 0 to 6897
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   posicion   6898 non-null   float64
 1   juego      6898 non-null   object 
 2   dinero     6898 non-null   object 
 3   jugadores  6898 non-null   object 
 4   torneos    6898 non-null   object 
 5   fecha      6898 non-null   object 
dtypes: float64(1), object(5)
memory usage: 323.5+ KB


In [30]:
df["dinero"] = df["dinero"].str.lstrip("$").str.replace(",","").astype("float")

In [49]:
df["torneos"] = df["torneos"].str.rstrip(" Tournaments")
df["torneos"] = df["torneos"].str.rstrip(" Tournament")
df["torneos"] = df["torneos"].astype("int")


In [42]:
df.head(1)

,posicion,juego,dinero,jugadores,torneos,fecha
0,1.0,StarCraft II,89399.27,105,53,2012-01-01


In [47]:
df["jugadores"] = df["jugadores"].str.rstrip(" Players")
df["jugadores"] = df["jugadores"].str.rstrip(" Player")
df["jugadores"] = df["jugadores"].astype("int")

In [43]:
df.head(1)

,posicion,juego,dinero,jugadores,torneos,fecha
0,1.0,StarCraft II,89399.27,105,53,2012-01-01


In [44]:
df["fecha"] = pd.to_datetime(df["fecha"])

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6898 entries, 0 to 6897
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   posicion   6898 non-null   float64       
 1   juego      6898 non-null   object        
 2   dinero     6898 non-null   float64       
 3   jugadores  6898 non-null   int32         
 4   torneos    6898 non-null   int32         
 5   fecha      6898 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int32(2), object(1)
memory usage: 269.6+ KB


In [51]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
posicion,6898.0,30.163671,2.042351e+01,1.0,14.0,27.000,44.0000,123.00
dinero,6898.0,205196.193778,1.145565e+06,0.0,924.0,9361.395,82594.9075,40138651.26
jugadores,6898.0,64.547550,1.365561e+02,0.0,6.0,17.000,59.0000,1719.00
torneos,6898.0,7.163671,1.282323e+01,1.0,1.0,2.000,7.0000,172.00


In [52]:
df.to_csv("Tourneys.csv")